In [ ]:
!pip install dgl

     |████████████████████████████████| 4.4MB 8.3MB/s 


In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


In [ ]:
import dgl.function as fn

class SAGEConv(nn.Module):
    
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)
    
    def forward(self, g, h):
       
        with g.local_scope():
            g.srcdata['h'] = h
            # update_all is a message passing API.
            g.update_all(fn.copy_u('h', 'm'), fn.mean('m', 'h_neigh'))
            h_neigh = g.dstdata['h_neigh']
            h_total = torch.cat([h_dst, h_neigh], dim=1)
            return F.relu(self.linear(h_total))

In [ ]:
class SAGEConv(nn.Module):
    
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)
    
    def forward(self, g, h, w):
        
        h_dst = h[:g.number_of_dst_nodes()]
        with g.local_scope():
            g.srcdata['h'] = h
            g.edata['w'] = w
            # update_all is a message passing API.
            g.update_all(fn.u_mul_e('h', 'w', 'm'), fn.mean('m', 'h_neigh'))
            h_neigh = g.dstdata['h_neigh']
            h_total = torch.cat([h_dst, h_neigh], dim=1)
            return F.relu(self.linear(h_total))

In [ ]:

def u_mul_e_udf(edges):
  
    return {'m' : edges.src['h'] * edges.data['w']}